In [1]:
import pandas as pd
import re
import numpy as np
import nltk 
import math
from nltk.corpus import stopwords 

stop_words = set(stopwords.words('english'))
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Siddh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [74]:
ds = pd.read_csv("db.csv", header=None)
ds.columns = ["Title", "Company", "Location", "Type", "Link"]

ds.head()

,Title,Company,Location,Type,Link
0,Teaching Faculty,SONAM TUITIONS,"Kalyan, Maharashtra",Flexible shift,https://in.indeed.com/pagead/clk?mo=r&ad=-6NYl...
1,Norwegian content writer required,Great Ponton,"Mumbai, Maharashtra",Day shift,https://in.indeed.com/pagead/clk?mo=r&ad=-6NYl...
2,MBA (Marketing),Astrotel Pvt Ltd,"Kanpur, Uttar Pradesh",Full-time,https://in.indeed.com/pagead/clk?mo=r&ad=-6NYl...
3,Visa counsellor,VISA SPOT,"Mohali, Punjab",Full-time,https://in.indeed.com/pagead/clk?mo=r&ad=-6NYl...
4,Scrum Master 4,Comcast,"Chennai, Tamil Nadu",Weekend availability,https://in.indeed.com/pagead/clk?mo=r&ad=-6NYl...


In [75]:
sample = pd.read_csv("naukri.csv")

sample.head()

,job_title,role,industry
0,Digital Media Planner,Advertising,"Advertising, PR, MR, Event Management"
1,Online Bidding Executive,Retail Sales,"IT-Software, Software Services"
2,Trainee Research/ Research Executive- Hi- Tec...,R&D,"Recruitment, Staffing"
3,Technical Support,Admin/Maintenance/Security/Datawarehousing,"IT-Software, Software Services"
4,Software Test Engineer -hyderabad,Programming & Design,"IT-Software, Software Services"


In [76]:
sample = sample.dropna()
sample.industry = sample.industry.apply(lambda x: re.split(',| ', x)[0])
sample.head()

,job_title,role,industry
0,Digital Media Planner,Advertising,Advertising
1,Online Bidding Executive,Retail Sales,IT-Software
2,Trainee Research/ Research Executive- Hi- Tec...,R&D,Recruitment
3,Technical Support,Admin/Maintenance/Security/Datawarehousing,IT-Software
4,Software Test Engineer -hyderabad,Programming & Design,IT-Software


In [77]:
industry_counts = sample['industry'].value_counts()

popular_industries = industry_counts[industry_counts>5]

popular_industries_list = popular_industries.index.tolist()

sample = sample[sample['industry'].isin(popular_industries_list)]

sample.head()

,job_title,role,industry
0,Digital Media Planner,Advertising,Advertising
1,Online Bidding Executive,Retail Sales,IT-Software
2,Trainee Research/ Research Executive- Hi- Tec...,R&D,Recruitment
3,Technical Support,Admin/Maintenance/Security/Datawarehousing,IT-Software
4,Software Test Engineer -hyderabad,Programming & Design,IT-Software


In [64]:
import sklearn.metrics as mt

from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [78]:
X = sample["job_title"]
Y = sample["industry"]

from sklearn.model_selection import train_test_split as tt
X_train, X_test, Y_train, Y_test = tt(X, Y, test_size=0.2, random_state=100)

In [79]:
vec = CountVectorizer(stop_words='english')
vec.fit(X_train)

CountVectorizer(stop_words='english')

In [80]:
X_train_transformed = vec.transform(X_train)
X_test_transformed = vec.transform(X_test)

In [81]:
model = MultinomialNB()
model.fit(X_train_transformed, Y_train)

MultinomialNB()

In [82]:
y_pred = model.predict(X_test_transformed)
y_pred_prob = model.predict_proba(X_test_transformed)

In [84]:
print(mt.classification_report(Y_test, y_pred, zero_division=1))

                precision    recall  f1-score   support

    Accounting       0.40      0.05      0.09        80
   Advertising       0.50      0.05      0.09        97
   Agriculture       1.00      0.00      0.00        10
     Animation       1.00      0.00      0.00         2
  Architecture       1.00      0.00      0.00        25
    Automobile       0.53      0.09      0.16        96
      Aviation       1.00      0.00      0.00         5
           BPO       0.56      0.60      0.58       377
       Banking       0.42      0.48      0.45       330
      Ceramics       1.00      0.00      0.00         5
     Chemicals       0.50      0.04      0.08        23
  Construction       0.45      0.08      0.13       117
      Consumer       1.00      0.00      0.00        30
       Courier       1.00      0.00      0.00        33
     Education       0.67      0.40      0.50       226
   Electricals       1.00      0.00      0.00        22
        Export       1.00      0.00      0.00  

In [85]:
# vec = CountVectorizer(stop_words="english")

# vec.fit(ds.Title)
X_test_transformed = vec.transform(ds.Title)

In [86]:
ds

,Title,Company,Location,Type,Link
0,Teaching Faculty,SONAM TUITIONS,"Kalyan, Maharashtra",Flexible shift,https://in.indeed.com/pagead/clk?mo=r&ad=-6NYl...
1,Norwegian content writer required,Great Ponton,"Mumbai, Maharashtra",Day shift,https://in.indeed.com/pagead/clk?mo=r&ad=-6NYl...
2,MBA (Marketing),Astrotel Pvt Ltd,"Kanpur, Uttar Pradesh",Full-time,https://in.indeed.com/pagead/clk?mo=r&ad=-6NYl...
3,Visa counsellor,VISA SPOT,"Mohali, Punjab",Full-time,https://in.indeed.com/pagead/clk?mo=r&ad=-6NYl...
4,Scrum Master 4,Comcast,"Chennai, Tamil Nadu",Weekend availability,https://in.indeed.com/pagead/clk?mo=r&ad=-6NYl...
...,...,...,...,...,...
3952,EY - Director - Cyber Security - Government Ad...,EY India,Delhi,Regular,https://www.shine.com/jobs/ey-director-cyber-s...
3953,SafexPay - Head - Technology,Paygate India Private Limited,Mumbai City,Regular,https://www.shine.com/jobs/safexpay-head-techn...
3954,Process Developer - Accounts Payable,Genpact India Pvt. Ltd.,Kolkata,Regular,https://www.shine.com/jobs/process-developer-a...
3955,Continuum Green Energy - Company Secretary,Continuum Green Energy (India) Pvt ...,Mumbai City,Regular,https://www.shine.com/jobs/continuum-green-ene...


In [87]:
roles = model.predict(X_test_transformed)

ds["role"] = roles 

ds

,Title,Company,Location,Type,Link,role
0,Teaching Faculty,SONAM TUITIONS,"Kalyan, Maharashtra",Flexible shift,https://in.indeed.com/pagead/clk?mo=r&ad=-6NYl...,Education
1,Norwegian content writer required,Great Ponton,"Mumbai, Maharashtra",Day shift,https://in.indeed.com/pagead/clk?mo=r&ad=-6NYl...,IT-Software
2,MBA (Marketing),Astrotel Pvt Ltd,"Kanpur, Uttar Pradesh",Full-time,https://in.indeed.com/pagead/clk?mo=r&ad=-6NYl...,IT-Software
3,Visa counsellor,VISA SPOT,"Mohali, Punjab",Full-time,https://in.indeed.com/pagead/clk?mo=r&ad=-6NYl...,Education
4,Scrum Master 4,Comcast,"Chennai, Tamil Nadu",Weekend availability,https://in.indeed.com/pagead/clk?mo=r&ad=-6NYl...,IT-Software
...,...,...,...,...,...,...
3952,EY - Director - Cyber Security - Government Ad...,EY India,Delhi,Regular,https://www.shine.com/jobs/ey-director-cyber-s...,Banking
3953,SafexPay - Head - Technology,Paygate India Private Limited,Mumbai City,Regular,https://www.shine.com/jobs/safexpay-head-techn...,IT-Software
3954,Process Developer - Accounts Payable,Genpact India Pvt. Ltd.,Kolkata,Regular,https://www.shine.com/jobs/process-developer-a...,IT-Software
3955,Continuum Green Energy - Company Secretary,Continuum Green Energy (India) Pvt ...,Mumbai City,Regular,https://www.shine.com/jobs/continuum-green-ene...,Recruitment


In [126]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [168]:
ds['combined'] = ds['Title'] + ' ' + ds['role'] + ' ' + ds['Location']

In [169]:
tfIdfVectorizer=TfidfVectorizer(use_idf=True)
tfIdf = tfIdfVectorizer.fit_transform(ds['combined'].tolist())

In [189]:
inputs = {
    'Title': 'Software Development', 
    'Location': 'Chennai'
}

usertitle = vec.transform([inputs['Title']])
userrole = model.predict(usertitle)

user = inputs['Title'] + ' ' + inputs['Location'] + ' ' + userrole[0]

In [190]:
input_vector = tfIdfVectorizer.transform([user])

cosine_similarities = cosine_similarity(input_vector, tfIdf).flatten()

most_similar_indices = cosine_similarities.argsort()[::-1]

In [191]:
for index in most_similar_indices:
    print("Title: '{}' '{}' '{}' \t Cosine Similarity: {}".format(ds.loc[index, 'Title'], ds.loc[index, 'Company'], ds.loc[index, 'Location'], cosine_similarities[index]))

Title: 'Engineer II - Software Development' 'Verizon Data Services India Pvt.Ltd' 'Chennai' 	 Cosine Similarity: 0.7174477521094883
Title: 'Engineering III Consultant-Software Development' 'Verizon Data Services India Pvt.Ltd' 'Chennai' 	 Cosine Similarity: 0.6012361904415235
Title: 'Business Development Manager' 'A-CLASS MARBLE INDIA PRIVATE LIMITE...' 'Delhi' 	 Cosine Similarity: 0.5971407645530474
Title: 'Business Development' 'AIGM ALL INDIA GIFTS MANAGER' 'Mumbai City' 	 Cosine Similarity: 0.5855620320169513
Title: 'Product Development Manager' 'Geeta Aluminium Company Pvt. Ltd.' 'Mumbai, Maharashtra, India' 	 Cosine Similarity: 0.5419656734980483
Title: 'Urgent | Business Development Manager' 'BANGALORE JOBHUNTERR INDIA PRIVATE ...' 'Bangalore' 	 Cosine Similarity: 0.5320096989926753
Title: 'Deputy Manager - Learning & Development(1)' 'PropTiger.com' 'Chennai, Tamil Nadu, India' 	 Cosine Similarity: 0.5181674560123821
Title: 'Manager - Grants & Organisational Development - India 